In [1]:
!pip install mediapipe opencv-python

In [5]:
import cv2
import mediapipe as mp
import numpy as np
import time

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

def calculate_angle(a, b, c):
    a = np.array(a) # First point
    b = np.array(b) # Mid point
    c = np.array(c) # End point
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle > 180.0:
        angle = 360-angle
        
    return angle

# Function to create rounded rectangle
def draw_rounded_rectangle(img, top_left, bottom_right, radius, color, thickness=-1):
    x1, y1 = top_left
    x2, y2 = bottom_right
    
    # Draw the filled rectangle without corners
    cv2.rectangle(img, (x1 + radius, y1), (x2 - radius, y1 + radius), color, thickness)  # Top
    cv2.rectangle(img, (x1, y1 + radius), (x2, y2 - radius), color, thickness)  # Middle
    cv2.rectangle(img, (x1 + radius, y2 - radius), (x2 - radius, y2), color, thickness)  # Bottom
    
    # Draw the four corner circles
    cv2.circle(img, (x1 + radius, y1 + radius), radius, color, thickness)  # Top-left
    cv2.circle(img, (x2 - radius, y1 + radius), radius, color, thickness)  # Top-right
    cv2.circle(img, (x1 + radius, y2 - radius), radius, color, thickness)  # Bottom-left
    cv2.circle(img, (x2 - radius, y2 - radius), radius, color, thickness)  # Bottom-right

cap = cv2.VideoCapture(0)

# Squat counter variables
counter = 0 
stage = None
squat_depth = 0  # Track depth as percentage
feedback = ""
max_depth_reached = 0  # Track maximum depth reached during a squat

# UI parameters
panel_width = 250
panel_height = 100
corner_radius = 20
bg_color = (30, 30, 30)  # Dark background for minimalist look
text_color = (255, 255, 255)  # White text
accent_color = (0, 200, 255)  # Cyan accent
progress_color = (0, 255, 200)  # Teal for progress

# Debug info
debug_mode = False  # Set to True to show angle values

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame")
            break
            
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates for squat detection - using both legs for better accuracy
            # Right side
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            
            # Left side
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            
            # Calculate angles - average both sides for better accuracy
            right_knee_angle = calculate_angle(right_hip, right_knee, right_ankle)
            left_knee_angle = calculate_angle(left_hip, left_knee, left_ankle)
            knee_angle = (right_knee_angle + left_knee_angle) / 2
            
            right_hip_angle = calculate_angle(right_shoulder, right_hip, right_knee)
            left_hip_angle = calculate_angle(left_shoulder, left_hip, left_knee)
            hip_angle = (right_hip_angle + left_hip_angle) / 2
            
            # Convert knee angle to a 0-100% squat depth
            # 170° = standing straight (0%), 90° = deep squat (100%)
            # Adjusted range to make it more sensitive
            squat_depth = min(100, max(0, (170 - knee_angle) * 100 / 80))
            
            # Debug info
            if debug_mode:
                cv2.putText(image, f"Knee: {int(knee_angle)}", (10, 30), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
                cv2.putText(image, f"Hip: {int(hip_angle)}", (10, 60), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
                cv2.putText(image, f"Stage: {stage}", (10, 90), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
            
            # Track max depth in current squat
            if stage == "down":
                max_depth_reached = max(max_depth_reached, squat_depth)
            
            # IMPROVED SQUAT COUNTER LOGIC
            # Standing position
            if knee_angle > 150 and hip_angle > 160:
                # If we were in down position and reached sufficient depth, count the rep
                if stage == 'down' and max_depth_reached > A40:
                    stage = "up"
                    counter += 1
                    feedback = f"Rep counted! Depth: {int(max_depth_reached)}%"
                    max_depth_reached = 0  # Reset for next rep
                elif stage != 'down':
                    stage = "up"
                    feedback = "Ready"
                    max_depth_reached = 0  # Reset when standing
            
            # Squat position - more sensitive to detect going down
            elif knee_angle < 140:
                if stage == 'up' or stage is None:
                    stage = "down"
                feedback = f"Depth: {int(squat_depth)}%"
                    
        except:
            pass
        
        # Create minimalist UI panel at bottom of screen
        h, w, _ = image.shape
        panel_x = (w - panel_width) // 2  # Center horizontally
        panel_y = h - panel_height - 20  # Bottom with margin
        
        # Draw semi-transparent background
        overlay = image.copy()
        draw_rounded_rectangle(
            overlay, 
            (panel_x, panel_y), 
            (panel_x + panel_width, panel_y + panel_height), 
            corner_radius, 
            bg_color, 
            -1
        )
        
        # Apply overlay with transparency
        alpha = 0.8
        cv2.addWeighted(overlay, alpha, image, 1 - alpha, 0, image)
        
        # Draw depth gauge - horizontal bar
        gauge_height = 6
        gauge_y = panel_y + panel_height - 20
        gauge_width = panel_width - 40
        
        # Background of gauge
        cv2.rectangle(image, 
                     (panel_x + 20, gauge_y), 
                     (panel_x + 20 + gauge_width, gauge_y + gauge_height), 
                     (80, 80, 80), -1, cv2.LINE_AA)
        
        # Fill based on current depth
        filled_width = int((squat_depth / 100) * gauge_width)
        cv2.rectangle(image, 
                     (panel_x + 20, gauge_y), 
                     (panel_x + 20 + filled_width, gauge_y + gauge_height), 
                     progress_color, -1, cv2.LINE_AA)
        
        # First draw "SQUATS" label
        cv2.putText(image, "SQUATS", 
                   (panel_x + panel_width//2 - 35, panel_y + 25), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, text_color, 1, cv2.LINE_AA)
        
        # Counter in smaller font and placed below the "SQUATS" label
        cv2.putText(image, str(counter), 
                   (panel_x + panel_width//2 - (15 if counter < 10 else 25), panel_y + 55), 
                   cv2.FONT_HERSHEY_SIMPLEX, 1.2, text_color, 2, cv2.LINE_AA)
        
        # Current state and feedback
        if feedback:
            cv2.putText(image, feedback, 
                       (panel_x + 20, panel_y + 85), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, accent_color, 1, cv2.LINE_AA)
        
        # Simplified pose landmarks - just show key points for squats
        custom_connections = [
            (mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.LEFT_HIP),
            (mp_pose.PoseLandmark.RIGHT_SHOULDER, mp_pose.PoseLandmark.RIGHT_HIP),
            (mp_pose.PoseLandmark.LEFT_HIP, mp_pose.PoseLandmark.LEFT_KNEE),
            (mp_pose.PoseLandmark.RIGHT_HIP, mp_pose.PoseLandmark.RIGHT_KNEE),
            (mp_pose.PoseLandmark.LEFT_KNEE, mp_pose.PoseLandmark.LEFT_ANKLE),
            (mp_pose.PoseLandmark.RIGHT_KNEE, mp_pose.PoseLandmark.RIGHT_ANKLE),
            (mp_pose.PoseLandmark.LEFT_HIP, mp_pose.PoseLandmark.RIGHT_HIP),
        ]
        
        if results.pose_landmarks:
            # Draw only key landmarks
            for landmark_id in [
                mp_pose.PoseLandmark.LEFT_SHOULDER,
                mp_pose.PoseLandmark.RIGHT_SHOULDER,
                mp_pose.PoseLandmark.LEFT_HIP,
                mp_pose.PoseLandmark.RIGHT_HIP,
                mp_pose.PoseLandmark.LEFT_KNEE,
                mp_pose.PoseLandmark.RIGHT_KNEE,
                mp_pose.PoseLandmark.LEFT_ANKLE,
                mp_pose.PoseLandmark.RIGHT_ANKLE
            ]:
                landmark = results.pose_landmarks.landmark[landmark_id.value]
                h, w, c = image.shape
                cx, cy = int(landmark.x * w), int(landmark.y * h)
                # Draw larger circles for key points
                cv2.circle(image, (cx, cy), 7, accent_color, -1)
            
            # Draw custom connections
            for connection in custom_connections:
                start_point = results.pose_landmarks.landmark[connection[0].value]
                end_point = results.pose_landmarks.landmark[connection[1].value]
                
                start_x, start_y = int(start_point.x * w), int(start_point.y * h)
                end_x, end_y = int(end_point.x * w), int(end_point.y * h)
                
                cv2.line(image, (start_x, start_y), (end_x, end_y), progress_color, 3)
        
        cv2.imshow('Squat Tracker', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()